In [ ]:
import os
import sys
from pathlib import Path
import pandas as pd
#import json

%matplotlib inline

In [ ]:
sys.path.append(r"../LUCinSA_helpers")
from rf import *

In [ ]:
'''
PARAMETERS: modify in notebook_params notebook, then run that notebook and this cell to update here
DO NOT modify this cell
'''

%store -r basic_config
print(" modelling year is (filter_year): {} (this is first year if season spans two years)".format(basic_config['filter_yr']))
print("sample point file is: {}".format(basic_config['ptfile']))
%store -r timeseries_params
if timeseries_params['load_samp'] == False:
    print('using polygon file to make new point file: {}'.format(basic_config['polyfile']) )
    print('will sample {} pts per polygon'.format(timeseries_params['npts']))
%store -r classification_params
print("Classification_Params: \n" 
      " temp output files are saved to (local_model_dir): {} \n" 
      " shared modelling files are in (main_model_dir): {} \n" 
      " feature_model = {} \n "
      " feature_model_dict is: {} \n"
      " spec_indices = {} \n si_vars = {} \n singleton_vars = {} \n poly_vars = {}"
      .format(classification_params['local_model_dir'],classification_params['main_model_dir'],
              classification_params['feature_model'],
              classification_params['feature_mod_dict'], 
              classification_params['spec_indices'],classification_params['si_vars'],classification_params['singleton_vars'],classification_params['poly_vars']
              ))

### Check feature model settings
##### IF new feature model: Make sure spec_indices, si_vars, singleton_vars and poly_vars are set correctly in parameters (if not, set and rerun above)

## Load feature model info (if existing) or save feature model info (if new)

In [ ]:
#mod_dict = '../Feature_Models.json'
mod_dict = classification_params['feature_mod_dict']
spec_indices, si_vars,singleton_vars,poly_vars = getset_variable_model(mod_dict, 
                      classification_params['feature_model'], 
                      classification_params['spec_indices'], 
                      classification_params['si_vars'], 
                      classification_params['singleton_vars'],
                      classification_params['poly_vars']
                                                                      )

### Make variable dataframe (use all sample points initially -- can then reduce in Notebook 6b)
#### Note: This is pretty heavy and should be run from SLURM with bash script (var_data_frame.sh)


In [ ]:
in_dir = os.path.join(basic_config['smooth_dir'],'003737')
make_variable_stack(in_dir,
                    classification_params['feature_model'],
                    basic_config['filter_yr'],
                    classification_params['spec_indices'],
                    classification_params['si_vars'],
                    classification_params['feature_mod_dict'],
                    classification_params['singleton_vars'] ,
                    classification_params['singleton_var_dict'],
                    classification_params['poly_vars'], 
                    classification_params['poly_var_path'], 
                    None)

In [ ]:
'''
make_var_dataframe(classification_params['local_model_dir'],
                  classification_params['spec_indices'],
                  classification_params['si_vars'],
                  basic_config['smooth_dir'],
                  basic_config['grid_file'],
                  basic_config['grid_cells'],
                  basic_config['polyfile'],
                  oldest=timeseries_params['oldest_samp'],
                  newest=timeseries_params['newest_samp'],
                  npts=timeseries_params['npts'], 
                  seed=timeseries_params['seed1'],
                  load_samp=timeseries_params['load_samp'], 
                  ptfile=basic_config['ptfile'])
'''


## to augment sample with polygon data:

* join sample points to polygon data to get subset of polygons with points (join class info to those polygons)
* run make_var_dataframe with load_samp = False and 
    polygon file set to subset above-- will get {npts} random points from each polygon

## To save an html copy of this notebook with all outputs:
(these two cells should be last in notebook)

In [ ]:
%%script echo skipping
### comment out above line and run this cell to print output as html

out_name = str(basic_config['country']+'6a_RandomForest_FeathreModel_to_dataframe'+'_model'+str(classification_params['feature_model'])+str(basic_config['filter_yr']))
!jupyter nbconvert --output-dir='./Outputs' --to html --no-input --ExecutePreprocessor.store_widget_state=True --output=$out_name 6b_RandomFoest_ModelComparisons.ipynb